In [10]:
from IPython.display import display, Markdown
import re
from itertools import chain
import copy
import subprocess

In [11]:
def style_attributes(md):
    
    md_ = copy.deepcopy(md)
    # split markdown object
    delims = [
        '\n\n# ', 
        '\n\nParameters\n----------\n', 
        '\n\nReturns\n----------\n', 
        '\n\nReturns\n-------\n']
    for delim in delims:
        if isinstance(md, str):
            md = md.split(delim)
        if isinstance(md, list):   
            md = [m.split(delim) for m in md]   
            if isinstance(md[0], list):
                md = list(chain(*md))        
                
    # create dict with new style for each variable of each function
    repl = []
    for m in md:
        r = re.findall('\w+\W: ', m)
        if len(r):
            for r_s in r:
                n = m.split(r_s)
                n_1 = n[1]
                v = r_s.split(' : ')[0] # variable
                t = n_1.split('\n    ')[0] # type
                new = '> + ###### `{}` : {}\n'.format(v, t)
                old = '{}{}\n'.format(r_s, t)
                dic = {'old': old, 'new': new}
                repl.append(dic)    
                
    # apply the new style in the markdown object and save
    for rp in repl:
        md_ = md_.replace(rp['old'], rp['new'])
        
    return md_

def style_parameters_returns(md):

    md = md.replace('\nParameters\n----------\n', '\n> #### Parameters\n')
    md = md.replace('\nReturns\n----------\n', '\n> #### Returns\n')  
    md = md.replace('\nReturns\n-------\n', '\n> #### Returns\n')
    
    return md

def style_return_types(md):
    md_ = copy.deepcopy(md)
    aa = md_.split('> #### Returns\n')
    bb = []
    for a in aa:
        b = a.split('\n\n## ')
        if len(b) == 1:
            b = a.split('\n\n### ')        
        bb.append(b[0])

    repl = []
    for c in bb:
        try:
            d = c.split('\n    ')
            f = [e.split('\n') for e in d]
            ff = [item for sublist in f for item in sublist]
            types = ff[::2]
            descr = ff[1::2]
            gg = []
            for v in zip(types, descr):
                new = '> + ###### {}\n{}'.format(v[0], v[1])
                gg.append(new)
            if len(gg):
                old = c
                dic = {'old': old, 'new': ('\n').join(gg)}
                repl.append(dic)
        except:
            continue

    # apply the new style in the markdown object and save
    for rp in repl:
        md_ = md_.replace(rp['old'], rp['new'])
    
    if md_.endswith('\n> + ###### \n'):
        md_ = md_[:-13]
    return md_

In [12]:
parent = 'topojson'
children = [
    'topojson.core.topology', 
    'topojson.core.extract', 
    'topojson.core.join', 
    'topojson.core.cut', 
    'topojson.core.dedup',
    'topojson.core.hashmap', 
    'topojson.utils', 
    'topojson.ops'
]

In [13]:
# rp = subprocess.run(['pydocmd', 'simple', '{}++'.format(parent)], capture_output=True)
# md = rp.stdout.decode()

# pt_head = """---
# layout: default
# title: API reference
# nav_order: 3
# has_children: true
# permalink: /api-reference
# --- 
# """

# md_out = '{}\n\n{}'.format(pt_head, md)

# with open('docs/api/api-reference.md'.format(parent), 'w') as fp_out:
#     print(md_out, file=fp_out)

In [14]:
for idx, child in enumerate(children):
    rc = subprocess.run(['pydocmd', 'simple', '{}++'.format(child)], capture_output=True, cwd='../')
    md = rc.stdout.decode()
    
    ch_head = f'''---
layout: default
title: {child}
parent: API reference
nav_order: {idx+1}
---
'''
    
    md_out = style_attributes(md)
    md_out = style_parameters_returns(md_out)
    md_out = style_return_types(md_out)
    md_out = '{}\n\n{}'.format(ch_head, md_out)
    
    with open('../docs/api/{}.md'.format(child), 'w') as fp_out:
        print(md_out, file=fp_out)

In [ ]:
#display(Markdown(style_attributes(md)))